In [32]:
from glob import glob
import pickle
import pandas as pd
import geopandas as gpd

In [21]:
dt_lprov=glob('/data/ksa/05_Modelling/model/xgboost*_v2.pkl')

In [68]:
dict_acc_cluster_prov={}
for i in dt_lprov:
    prov=i.split('/')[-1].replace('xgboost','').replace('_v2.pkl','')
    dict_cluster={}
    with open(i,'rb') as file:
        dict_r=pickle.load(file)
        for j in dict_r.keys():
            vf1_m=dict_r[j]['validation_metrics']['val_f1_micro']
            vf1_ma=dict_r[j]['validation_metrics']['val_f1_macro']
            vac_fp=dict_r[j]['validation_metrics']['val_acc_fase_padi']
            dict_cluster[str(j)]={'f1micro':vf1_m,'f1macro':vf1_ma,'accpadi':vac_fp}
    dict_acc_cluster_prov[prov]=dict_cluster

In [69]:
com=pd.DataFrame()
for i in dict_acc_cluster_prov.keys():
    for j in dict_acc_cluster_prov[i].keys():
        x=pd.DataFrame.from_dict([dict_acc_cluster_prov[i][j]])
        x['kdprov']=i
        x['cluster_data']=str(j)
        com=pd.concat([com,x])

In [70]:
gl_peta=glob('/data/ksa/00_Data_Input/cluster_data/cluster_*_ok.gpkg')
x=gpd.GeoDataFrame()
for i in gl_peta:
    i=gpd.read_file(i)[['idkab','cluster_data','geometry']]
    i.cluster_data=i.cluster_data.astype(str)
    x=pd.concat([i,x])
x=gpd.GeoDataFrame(x,geometry=x['geometry']).fillna(0)
x['kdprov']=x.idkab.apply(lambda y: y[:2])
x['cluster_data']=x.apply(lambda y:'0' if y.kdprov=="94" else y.cluster_data,axis=1)

In [71]:
x=x.merge(com,how='outer')

In [75]:
x.query('kdprov=="35"')

,idkab,cluster_data,geometry,kdprov,f1micro,f1macro,accpadi
17,3522,0,"MULTIPOLYGON (((111.62266 -6.98469, 111.62268 ...",35,0.875190,0.843147,0.954555
18,3523,0,"MULTIPOLYGON (((111.70004 -6.75957, 111.70106 ...",35,0.875190,0.843147,0.954555
19,3524,0,"MULTIPOLYGON (((112.44671 -6.87406, 112.44681 ...",35,0.875190,0.843147,0.954555
57,3515,1,"MULTIPOLYGON (((112.84251 -7.33989, 112.84161 ...",35,0.883616,0.873478,0.903117
58,3525,1,"MULTIPOLYGON (((112.55698 -6.84752, 112.5573 -...",35,0.883616,0.873478,0.903117
59,3578,1,"MULTIPOLYGON (((112.66636 -7.19671, 112.66644 ...",35,0.883616,0.873478,0.903117
79,3506,2,"MULTIPOLYGON (((112.12258 -7.60695, 112.12268 ...",35,0.801883,0.770510,0.936244
80,3516,2,"MULTIPOLYGON (((112.44762 -7.30418, 112.44758 ...",35,0.801883,0.770510,0.936244
81,3517,2,"MULTIPOLYGON (((112.34422 -7.34464, 112.34515 ...",35,0.801883,0.770510,0.936244
82,3518,2,"MULTIPOLYGON (((112.04815 -7.39372, 112.04815 ...",35,0.801883,0.770510,0.936244


In [74]:
x.to_file('cluster_with_acc.gpkg')